### 1. Welcome to ArenaFlow Stadium Operations Hub

**Objective**: Dive into the ArenaFlow use case and discover how Dynamic Tables power real-time stadium operations.

**What You’ll Learn**:
- How Dynamic Tables handle high-velocity data for minute-by-minute insights.
- Steps to build a real-time pipeline for smarter operational decisions.
- Key Snowflake features in action: Dynamic Tables, Snowpipe, Streams, and Notebooks.

**Scenario**: Imagine a 50,000-seat stadium during a sold-out basketball game. Concession stands are packed, beer stocks are dwindling, and lines are growing. ArenaFlow transforms raw data from POS systems, IoT sensors, queue cameras, and staff apps into a live, minute-by-minute dashboard. This empowers managers to:
- **Redirect staff** to busy areas, cutting wait times and boosting sales.
- **Restock inventory** to keep fans supplied with favorites.
- **Optimize comfort** by tweaking HVAC based on temp and CO2 readings.
- **Fix issues** like spills or restroom needs for a seamless fan experience.

**Business Value**: Every minute matters in a stadium. Delays or stock-outs frustrate fans, hurting revenue and reviews. Dynamic Tables drive proactive operations, keeping fans happy, spending, and cheering. By the end, you’ll craft a pipeline that delivers these wins, showing Snowflake’s edge for venues.

**Key Features**:
- **Dynamic Tables**: SQL-based streaming with auto-refresh.
- **Snowpipe**: Continuous data loading.
- **Streams & Tasks**: Automated workflows.
- **Snowflake Notebooks**: SQL and Python collaboration hub.

**Next**: Set up your Snowflake environment to kick off the pipeline.

### 2. Connect to Your Snowflake Session

**Objective**: Establish a Snowpark session to interact with Snowflake.

**Context**: This Python code connects to your active Snowflake session and displays your current context—account, role, database, schema, and warehouse—for verification.

**Value**: A confirmed connection ensures you can execute SQL and Python commands to build the ArenaFlow pipeline, laying the groundwork for real-time data processing.

**Next**: Explore the ArenaFlow data model.

In [ ]:
from snowflake.snowpark.context import get_active_session

# establish a session
session = get_active_session()

# print the current session context
print(f"""
SESSION CONTEXT
  Account:   {session.get_current_account()}
  Role:      {session.get_current_role()}
  Database:  {session.get_current_database()}
  Schema:    {session.get_current_schema()}
  Warehouse: {session.get_current_warehouse()}
"""
)

### 3. Understanding the ArenaFlow Data Model

**Objective**: Grasp the data model powering ArenaFlow’s real-time insights.

**Overview**:
- **Static Reference Tables (Dimension Layer)**:
  - **Stand Directory**: Maps concession stands (e.g., “West Concourse”, food).
  - **Product Catalog**: Details SKUs (e.g., “Draft Beer 16oz”, 300-unit par).
  - **Sensor Map**: Links IoT sensors to zones (e.g., queue cams in “Lower Bowl A”).
  - **Maintenance Thresholds**: Sets alert rules (e.g., queue > 15 = warning).
  - Stable during events, these provide context for live data.
- **Dynamic Tables (Real-Time Layer)**:
  - **Raw Events**: Captures all incoming data (sales, inventory, queues).
  - **Clean Events**: Structures and validates data.
  - **Sales, Inventory, Queue Metrics**: Delivers minute-by-minute KPIs.
  - **Ops Alerts**: Flags issues for staff action.
  - **Arena Dashboard**: Unifies insights for managers.

**Value**: Static tables give consistent context, while Dynamic Tables process live data fast. Together, they enable decisions like rushing staff to busy stands or restocking beer, lifting fan satisfaction and sales (e.g., 5-min shorter queues can boost concessions 10-15%).

**Why It Matters**: A solid data model blends accuracy and speed for actionable, real-time operations.

**Next**: Build the static reference tables.

### 4. Building Static Reference Tables

**Objective**: Create static tables to enrich real-time event data.

**Context**: These tables—Stand Directory, Product Catalog, Sensor Map, and Maintenance Thresholds—provide metadata to make live data meaningful. For example:
- **Stand Directory**: Ties sales to locations (e.g., “STAND_001” = West Concourse, food).
- **Product Catalog**: Maps SKUs to stands and categories (e.g., “SKU_006” = Draft Beer, beverage).
- **Sensor Map**: Connects sensors to zones (e.g., “SENSOR_001” = queue cam, West Concourse).
- **Maintenance Thresholds**: Defines alerts (e.g., queue > 15 = warning, > 25 = critical).

**SQL Steps**:
1. **Stand Directory**: Creates a table with stand IDs, locations, and categories (food, beverage, merch).
2. **Product Catalog**: Links SKUs to stands, categories, and par levels (e.g., 300 units for beer).
3. **Sensor Map**: Maps sensors to zones and types (queue_cam, co2, temp, restroom).
4. **Maintenance Thresholds**: Sets rules for queues, stock, CO2, and temp with severity levels.
5. **QA Queries**: Each table includes a `SELECT *` to verify data.

**Value**: These tables ground your pipeline in reliable context. Knowing a stand’s location or a threshold for action automates issue detection, speeds decisions, and enhances fan experience—e.g., prioritizing staff to busy areas or flagging low stock.

**Next**: Simulate real-time stadium events.

In [ ]:
-- Stand Directory
CREATE TABLE IF NOT EXISTS STAND_DIRECTORY AS
SELECT
  stand_id,
  location,
  category
FROM (
  SELECT
    'STAND_001' AS stand_id, 'West Concourse' AS location, 'food' AS category
  UNION ALL
  SELECT
    'STAND_002' AS stand_id, 'East Concourse' AS location, 'beverage' AS category
  UNION ALL
  SELECT
    'STAND_003' AS stand_id, 'Lower Bowl A' AS location, 'merch' AS category
  UNION ALL
  SELECT
    'STAND_004' AS stand_id, 'Club Seats North' AS location, 'food' AS category
  UNION ALL
  SELECT
    'STAND_005' AS stand_id, 'Upper Deck South' AS location, 'beverage' AS category
);

In [ ]:
SELECT *
FROM STAND_DIRECTORY;

In [ ]:
CREATE OR REPLACE TABLE PRODUCT_CATALOG AS
SELECT DISTINCT
  sku_id,
  name,
  CASE
    WHEN sku_id IN ('SKU_001', 'SKU_002', 'SKU_003', 'SKU_004', 'SKU_005') THEN 'STAND_001'
    WHEN sku_id IN ('SKU_006', 'SKU_007', 'SKU_008', 'SKU_009', 'SKU_010') THEN 'STAND_002'
    ELSE 'STAND_003'
  END AS stand_id,
  CASE
    WHEN sku_id IN ('SKU_001', 'SKU_002', 'SKU_003', 'SKU_004', 'SKU_005') THEN 'food'
    WHEN sku_id IN ('SKU_006', 'SKU_007', 'SKU_008', 'SKU_009', 'SKU_010') THEN 'beverage'
    ELSE 'merch'
  END AS category,
  par_level_units
FROM (
  -- Food products
  SELECT 'SKU_001' AS sku_id, 'Cheeseburger' AS name, 150 AS par_level_units
  UNION ALL
  SELECT 'SKU_002', 'Hot Dog', 150
  UNION ALL
  SELECT 'SKU_003', 'Nachos', 100
  UNION ALL
  SELECT 'SKU_004', 'Pizza Slice', 120
  UNION ALL
  SELECT 'SKU_005', 'Fries', 200

  -- Beverage products
  UNION ALL
  SELECT 'SKU_006', 'Draft Beer 16oz', 300
  UNION ALL
  SELECT 'SKU_007', 'Soda 20oz', 250
  UNION ALL
  SELECT 'SKU_008', 'Bottled Water', 400
  UNION ALL
  SELECT 'SKU_009', 'Iced Tea', 200
  UNION ALL
  SELECT 'SKU_010', 'Coffee', 150

  -- Merch products
  UNION ALL
  SELECT 'SKU_011', 'Team Jersey', 50
  UNION ALL
  SELECT 'SKU_012', 'Cap', 75
  UNION ALL
  SELECT 'SKU_013', 'T-Shirt', 100
  UNION ALL
  SELECT 'SKU_014', 'Scarf', 60
  UNION ALL
  SELECT 'SKU_015', 'Keychain', 200
) products;

In [ ]:
SELECT *
FROM PRODUCT_CATALOG;

In [ ]:
-- Sensor Map
CREATE TABLE IF NOT EXISTS SENSOR_MAP AS
SELECT
  sensor_id,
  zone,
  sensor_type
FROM (
  SELECT 'SENSOR_001' AS sensor_id, 'West Concourse' AS zone, 'queue_cam' AS sensor_type
  UNION ALL
  SELECT 'SENSOR_002' AS sensor_id, 'East Concourse' AS zone, 'co2' AS sensor_type
  UNION ALL
  SELECT 'SENSOR_003' AS sensor_id, 'Lower Bowl A' AS zone, 'temp' AS sensor_type
  UNION ALL
  SELECT 'SENSOR_004' AS sensor_id, 'Club Seats North' AS zone, 'restroom' AS sensor_type
  UNION ALL
  SELECT 'SENSOR_005' AS sensor_id, 'Upper Deck South' AS zone, 'queue_cam' AS sensor_type
);

In [ ]:
SELECT *
FROM SENSOR_MAP;

In [ ]:
-- Maintenance Thresholds
CREATE TABLE IF NOT EXISTS MAINTENANCE_THRESHOLDS AS
SELECT
  metric_type,
  threshold_value,
  severity
FROM (
  SELECT 'queue_len' AS metric_type, 15.0 AS threshold_value, 'warning' AS severity
  UNION ALL
  SELECT 'queue_len' AS metric_type, 25.0 AS threshold_value, 'critical' AS severity
  UNION ALL
  SELECT 'stock_pct' AS metric_type, 0.3 AS threshold_value, 'warning' AS severity
  UNION ALL
  SELECT 'stock_pct' AS metric_type, 0.1 AS threshold_value, 'critical' AS severity
  UNION ALL
  SELECT 'co2_ppm' AS metric_type, 1200.0 AS threshold_value, 'warning' AS severity
  UNION ALL
  SELECT 'co2_ppm' AS metric_type, 1800.0 AS threshold_value, 'critical' AS severity
  UNION ALL
  SELECT 'temp_f' AS metric_type, 80.6 AS threshold_value, 'warning' AS severity
  UNION ALL
  SELECT 'temp_f' AS metric_type, 86.0 AS threshold_value, 'critical' AS severity
);

In [ ]:
SELECT *
FROM MAINTENANCE_THRESHOLDS;

### 5. Simulating Real-Time Stadium Events

**Objective**: Generate synthetic data to mimic live stadium activity.

**Why Simulate?**: Real stadiums stream data from POS, sensors, and cameras constantly. We’ll use synthetic data to replicate:
- **Sales**: Fans buying burgers or beers.
- **Inventory**: Stock dropping at stands.
- **Queues**: Lines growing at concessions.

This lands in `RAW_EVENTS`, mimicking Snowpipe’s real-time ingestion. Simulation lets you:
- Test pipeline speed without live feeds.
- Demo real-time processing safely.
- Learn Snowflake’s streaming power.

**Value**: Real-time data drives stadium success. A 10-minute delay in spotting a long line could cost hundreds in sales. Synthetic data preps your pipeline for live action, potentially boosting throughput 20% and revenue.

**Next**: Set up the `RAW_EVENTS` table.

### 6. Creating the Raw Events Table

**Objective**: Build the `RAW_EVENTS` table to capture all event data.

**Context**: This table uses a VARIANT column (`payload`) to store diverse events—sales, inventory, queues, environmental, and cleaning requests—mimicking Snowpipe’s continuous load every few seconds. The SQL generates:
- **Sales**: 100 rows, ~20/min, with SKUs, units, and prices.
- **Inventory**: 80 rows, ~16/min, with remaining units.
- **Queues**: 80 rows, ~16/min, with line lengths.
- **Environmental**: 100 rows, ~20/min, with temp and CO2.
- **Clean Requests**: 20 rows, ~4/min, with zones and severity.

**Value**: A unified table simplifies ingestion, scaling to handle peak game traffic. This keeps insights fresh, powering fast decisions to keep fans happy.

**Next**: Learn about the stored procedure for ongoing simulation.

In [ ]:
CREATE OR REPLACE TABLE RAW_EVENTS (
  event_id STRING,
  event_type STRING,
  event_ts TIMESTAMP_NTZ,
  stand_id STRING,
  sensor_id STRING,
  payload VARIANT,
  ingestion_time TIMESTAMP_NTZ
) AS
SELECT
  event_id,
  event_type,
  event_ts,
  stand_id,
  sensor_id,
  payload,
  ingestion_time
FROM (
  -- Sale events (100 rows, ~20 per minute)
  SELECT
    'EVT_SALE_INIT_' || SEQ8() AS event_id,
    'sale' AS event_type,
    DATEADD(SECOND, UNIFORM(0, 299, RANDOM()), TO_TIMESTAMP_NTZ('2025-05-20 12:00:00')) AS event_ts,
    CASE
      WHEN RANDOM() % 5 = 0 THEN 'STAND_001'
      WHEN RANDOM() % 5 = 1 THEN 'STAND_002'
      WHEN RANDOM() % 5 = 2 THEN 'STAND_003'
      WHEN RANDOM() % 5 = 3 THEN 'STAND_004'
      ELSE 'STAND_005'
    END AS stand_id,
    NULL AS sensor_id,
    OBJECT_CONSTRUCT(
      'sku_id', 
        CASE 
          WHEN stand_id IN ('STAND_001', 'STAND_004') THEN 'SKU_00' || CAST(MOD(SEQ8(), 5) + 1 AS STRING)
          WHEN stand_id IN ('STAND_002', 'STAND_005') THEN 'SKU_00' || CAST(MOD(SEQ8(), 5) + 6 AS STRING)
          ELSE 'SKU_0' || CAST(MOD(SEQ8(), 5) + 11 AS STRING)
        END,
      'item_name',
        CASE
          WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 'Cheeseburger'
            WHEN 1 THEN 'Hot Dog'
            WHEN 2 THEN 'Nachos'
            WHEN 3 THEN 'Pizza Slice'
            ELSE 'Fries'
          END
          WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 'Draft Beer 16oz'
            WHEN 1 THEN 'Soda 20oz'
            WHEN 2 THEN 'Bottled Water'
            WHEN 3 THEN 'Iced Tea'
            ELSE 'Coffee'
          END
          ELSE CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 'Team Jersey'
            WHEN 1 THEN 'Cap'
            WHEN 2 THEN 'T-Shirt'
            WHEN 3 THEN 'Scarf'
            ELSE 'Keychain'
          END
        END,
      'vendor_name', CASE stand_id
        WHEN 'STAND_001' THEN 'Burger Bonanza'
        WHEN 'STAND_002' THEN 'Thirsty Fan Brews'
        WHEN 'STAND_003' THEN 'Team Spirit Merch'
        WHEN 'STAND_004' THEN 'Pizza Pavilion'
        ELSE 'Sip & Go'
      END,
      'units', UNIFORM(1, 5, RANDOM()),
      'price', 
        CASE
          WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 10.0 -- Cheeseburger
            WHEN 1 THEN 8.0 -- Hot Dog
            WHEN 2 THEN 7.0 -- Nachos
            WHEN 3 THEN 6.0 -- Pizza Slice
            ELSE 5.0 -- Fries
          END
          WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 8.0 -- Draft Beer 16oz
            WHEN 1 THEN 4.0 -- Soda 20oz
            WHEN 2 THEN 3.0 -- Bottled Water
            WHEN 3 THEN 4.5 -- Iced Tea
            ELSE 5.0 -- Coffee
          END
          ELSE CASE MOD(SEQ8(), 5)
            WHEN 0 THEN 50.0 -- Team Jersey
            WHEN 1 THEN 25.0 -- Cap
            WHEN 2 THEN 20.0 -- T-Shirt
            WHEN 3 THEN 15.0 -- Scarf
            ELSE 5.0 -- Keychain
          END
        END
    ) AS payload,
    DATEADD(SECOND, 1, event_ts) AS ingestion_time
  FROM TABLE(GENERATOR(ROWCOUNT => 100))
  
  UNION ALL
  
  -- Inventory events (80 rows, ~16 per minute)
  SELECT
    'EVT_INV_INIT_' || SEQ8() AS event_id,
    'inventory' AS event_type,
    DATEADD(SECOND, UNIFORM(0, 299, RANDOM()), TO_TIMESTAMP_NTZ('2025-05-20 12:00:00')) AS event_ts,
    CASE
      WHEN RANDOM() % 5 = 0 THEN 'STAND_001'
      WHEN RANDOM() % 5 = 1 THEN 'STAND_002'
      WHEN RANDOM() % 5 = 2 THEN 'STAND_003'
      WHEN RANDOM() % 5 = 3 THEN 'STAND_004'
      ELSE 'STAND_005'
    END AS stand_id,
    NULL AS sensor_id,
    OBJECT_CONSTRUCT(
      'sku_id',
        CASE 
          WHEN stand_id IN ('STAND_001', 'STAND_004') THEN 'SKU_00' || CAST(MOD(SEQ8(), 5) + 1 AS STRING)
          WHEN stand_id IN ('STAND_002', 'STAND_005') THEN 'SKU_00' || CAST(MOD(SEQ8(), 5) + 6 AS STRING)
          ELSE 'SKU_0' || CAST(MOD(SEQ8(), 5) + 11 AS STRING)
        END,
      'item_name', CASE
        WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 'Cheeseburger'
          WHEN 1 THEN 'Hot Dog'
          WHEN 2 THEN 'Nachos'
          WHEN 3 THEN 'Pizza Slice'
          ELSE 'Fries'
        END
        WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 'Draft Beer 16oz'
          WHEN 1 THEN 'Soda 20oz'
          WHEN 2 THEN 'Bottled Water'
          WHEN 3 THEN 'Iced Tea'
          ELSE 'Coffee'
        END
        ELSE CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 'Team Jersey'
          WHEN 1 THEN 'Cap'
          WHEN 2 THEN 'T-Shirt'
          WHEN 3 THEN 'Scarf'
          ELSE 'Keychain'
        END
      END,
      'vendor_name', CASE stand_id
        WHEN 'STAND_001' THEN 'Burger Bonanza'
        WHEN 'STAND_002' THEN 'Thirsty Fan Brews'
        WHEN 'STAND_003' THEN 'Team Spirit Merch'
        WHEN 'STAND_004' THEN 'Pizza Pavilion'
        ELSE 'Sip & Go'
      END,
      'units_remaining', UNIFORM(75, 100, RANDOM()) * 0.01 * CASE
        WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 150 -- Cheeseburger max
          WHEN 1 THEN 200 -- Hot Dog max
          WHEN 2 THEN 120 -- Nachos max
          WHEN 3 THEN 180 -- Pizza Slice max
          ELSE 300 -- Fries max
        END
        WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 450 -- Draft Beer 16oz max
          WHEN 1 THEN 375 -- Soda 20oz max
          WHEN 2 THEN 600 -- Bottled Water max
          WHEN 3 THEN 300 -- Iced Tea max
          ELSE 225 -- Coffee max
        END
        ELSE CASE MOD(SEQ8(), 5)
          WHEN 0 THEN 75 -- Team Jersey max
          WHEN 1 THEN 110 -- Cap max
          WHEN 2 THEN 150 -- T-Shirt max
          WHEN 3 THEN 90 -- Scarf max
          ELSE 300 -- Keychain max
        END
      END
    ) AS payload,
    DATEADD(SECOND, 1, event_ts) AS ingestion_time
  FROM TABLE(GENERATOR(ROWCOUNT => 80))
  
  UNION ALL
  
  -- Queue events (80 rows, ~16 per minute)
  SELECT
    'EVT_QUEUE_INIT_' || SEQ8() AS event_id,
    'queue' AS event_type,
    DATEADD(SECOND, UNIFORM(0, 299, RANDOM()), TO_TIMESTAMP_NTZ('2025-05-20 12:00:00')) AS event_ts,
    CASE
      WHEN RANDOM() % 5 = 0 THEN 'STAND_001'
      WHEN RANDOM() % 5 = 1 THEN 'STAND_002'
      WHEN RANDOM() % 5 = 2 THEN 'STAND_003'
      WHEN RANDOM() % 5 = 3 THEN 'STAND_004'
      ELSE 'STAND_005'
    END AS stand_id,
    CASE
      WHEN stand_id = 'STAND_001' THEN 'SENSOR_001'
      WHEN stand_id = 'STAND_002' THEN 'SENSOR_002'
      WHEN stand_id = 'STAND_003' THEN 'SENSOR_003'
      WHEN stand_id = 'STAND_004' THEN 'SENSOR_004'
      ELSE 'SENSOR_005'
    END AS sensor_id,
    OBJECT_CONSTRUCT(
      'queue_len', CASE
        WHEN RANDOM() % 10 < 2 THEN UNIFORM(15, 30, RANDOM())
        ELSE UNIFORM(5, 15, RANDOM())
      END
    ) AS payload,
    DATEADD(SECOND, 1, event_ts) AS ingestion_time
  FROM TABLE(GENERATOR(ROWCOUNT => 80))
  
  UNION ALL
  
  -- Environmental events (100 rows, ~20 per minute)
  SELECT
    'EVT_ENV_INIT_' || SEQ8() AS event_id,
    'env' AS event_type,
    DATEADD(SECOND, UNIFORM(0, 299, RANDOM()), TO_TIMESTAMP_NTZ('2025-05-20 12:00:00')) AS event_ts,
    NULL AS stand_id,
    CASE
      WHEN RANDOM() % 5 = 0 THEN 'SENSOR_001'
      WHEN RANDOM() % 5 = 1 THEN 'SENSOR_002'
      WHEN RANDOM() % 5 = 2 THEN 'SENSOR_003'
      WHEN RANDOM() % 5 = 3 THEN 'SENSOR_004'
      ELSE 'SENSOR_005'
    END AS sensor_id,
    OBJECT_CONSTRUCT(
      'temp_f', ROUND(UNIFORM(64.4, 86.0, RANDOM()), 1),
      'co2_ppm', UNIFORM(600, 1600, RANDOM()),
      'zone', CASE
        WHEN sensor_id = 'SENSOR_001' THEN 'West Concourse'
        WHEN sensor_id = 'SENSOR_002' THEN 'East Concourse'
        WHEN sensor_id = 'SENSOR_003' THEN 'Lower Bowl A'
        WHEN sensor_id = 'SENSOR_004' THEN 'Club Seats North'
        ELSE 'Upper Deck South'
      END
    ) AS payload,
    DATEADD(SECOND, 1, event_ts) AS ingestion_time
  FROM TABLE(GENERATOR(ROWCOUNT => 100))
  
  UNION ALL
  
  -- Clean request events (20 rows, ~4 per minute)
  SELECT
    'EVT_CLEAN_INIT_' || SEQ8() AS event_id,
    'clean_req' AS event_type,
    DATEADD(SECOND, UNIFORM(0, 299, RANDOM()), TO_TIMESTAMP_NTZ('2025-05-20 12:00:00')) AS event_ts,
    NULL AS stand_id,
    NULL AS sensor_id,
    OBJECT_CONSTRUCT(
      'zone', CASE
        WHEN RANDOM() % 5 = 0 THEN 'West Concourse'
        WHEN RANDOM() % 5 = 1 THEN 'East Concourse'
        WHEN RANDOM() % 5 = 2 THEN 'Lower Bowl A'
        WHEN RANDOM() % 5 = 3 THEN 'Club Seats North'
        ELSE 'Upper Deck South'
      END,
      'severity', CASE
        WHEN RANDOM() % 2 = 0 THEN 'warning'
        ELSE 'critical'
      END,
      'description', CASE
        WHEN RANDOM() % 3 = 0 THEN 'Spill reported near seating area'
        WHEN RANDOM() % 3 = 1 THEN 'Restroom maintenance needed'
        ELSE 'Debris in concourse'
      END
    ) AS payload,
    DATEADD(SECOND, 1, event_ts) AS ingestion_time
  FROM TABLE(GENERATOR(ROWCOUNT => 20))
);

In [ ]:
-- QA
SELECT *
FROM RAW_EVENTS;

### 7. Stored Procedure: INSERT_ARENA_EVENTS_SINGLE

### Explanation of Stored Procedure `INSERT_ARENA_EVENTS_SINGLE` for Synthetic Data Generation

The stored procedure `INSERT_ARENA_EVENTS_SINGLE` is written in Python using Snowflake's Snowpark API to generate synthetic event data for a stadium or arena simulation. It inserts data into the `RAW_EVENTS` table in the `ARENAFLOW.DYNAMIC_TABLES` schema, simulating various types of events (sales, inventory updates, queue lengths, environmental conditions, and cleaning requests). Below is a detailed breakdown of how it generates this synthetic data:

#### 1. **Purpose and Structure**
- **Purpose**: The procedure simulates real-time events in a stadium by generating a batch of synthetic events for a single iteration, with each iteration representing a short time window (10 seconds). It tracks iterations using a table `ITERATION_TRACKER` to ensure events progress temporally.
- **Language and Runtime**: Written in Python (version 3.11) using the Snowflake Snowpark library, which allows programmatic interaction with Snowflake tables.
- **Handler**: The `main` function is the entry point for execution.

#### 2. **Key Components**

##### a. **Initialization**
- **Timestamp Management**:
  - Retrieves the maximum `event_ts` from the `RAW_EVENTS` table to determine the starting point for the current iteration's events.
  - If no events exist or an error occurs, defaults to `2025-05-20 12:00:00`.
  - Each iteration advances the timestamp by 10 seconds: `start_ts = max_ts + timedelta(seconds=((iteration - 1) * 10))`.
- **Iteration Tracking**:
  - Reads the current `iteration` from `ITERATION_TRACKER` and increments it by 1.
  - Updates `ITERATION_TRACKER` with the new iteration count and current timestamp at the end of the procedure.

##### b. **Event Volume**
- Randomly determines the number of rows for each event type per iteration:
  - **Sale events**: 8–16 rows.
  - **Inventory events**: 6–8 rows.
  - **Queue events**: 5–6 rows.
  - **Environmental events**: 5–6 rows.
  - **Clean request events**: 1–2 rows.
- Total rows per iteration vary between 25 and 38, simulating fluctuating activity levels.

##### c. **SQL Insert Statement**
- Constructs a dynamic SQL `INSERT` statement to populate the `RAW_EVENTS` table with synthetic data.
- Uses `UNION ALL` to combine five subqueries, one for each event type.
- Each subquery uses Snowflake’s `TABLE(GENERATOR(ROWCOUNT => n))` to create the specified number of rows.

#### 3. **Event Type Generation Details**

##### a. **Sale Events**
- **Fields**:
  - `event_id`: Format `EVT_SALE_{iteration}_<sequence>`, using `SEQ8()` for uniqueness.
  - `event_type`: Fixed as `'sale'`.
  - `event_ts`: Random timestamp within a 10-second window starting at `start_ts`.
  - `stand_id`: Randomly assigned from `STAND_001` to `STAND_005` using `SEQ8() % 5`.
  - `sensor_id`: `NULL`.
  - `payload`: JSON object containing:
    - `sku_id`: Determined by `stand_id`:
      - `STAND_001`, `STAND_004`: `SKU_001` to `SKU_005` (food).
      - `STAND_002`, `STAND_005`: `SKU_006` to `SKU_010` (beverages).
      - `STAND_003`: `SKU_011` to `SKU_015` (merchandise).
    - `item_name`: Maps to `sku_id` (e.g., `SKU_001` → `Cheeseburger`, `SKU_006` → `Draft Beer 16oz`).
    - `vendor_name`: Maps to `stand_id` (e.g., `STAND_001` → `Burger Bonanza`).
    - `units`: Randomly 1–5, but 5–10 every 10th iteration to simulate demand spikes.
    - `price`: Fixed based on `sku_id` (e.g., `Cheeseburger` → $10, `Bottled Water` → $3).
  - `ingestion_time`: `event_ts + 1 second`.
- **Logic**: Simulates customer purchases at various stands, with varying quantities and prices based on item type.

##### b. **Inventory Events**
- **Fields**:
  - `event_id`: Format `EVT_INV_{iteration}_<sequence>`.
  - `event_type`: `'inventory'`.
  - `event_ts`: Random within 10 seconds from `start_ts`.
  - `stand_id`: Randomly `STAND_001` to `STAND_005`.
  - `sensor_id`: `NULL`.
  - `payload`:
    - `sku_id`, `item_name`, `vendor_name`: Same mapping as sale events.
    - `units_remaining`: Calculated as a percentage of max inventory:
      - Normally 75–100% of max (e.g., `SKU_001` max is 150, so 112.5–150 units).
      - For `STAND_002` every 5th iteration and specific rows (`SEQ8() % 3 = 0`), drops to 50–75% of max to simulate low stock.
      - Max values are defined per SKU (e.g., `SKU_006` → 450, `SKU_011` → 75).
  - `ingestion_time`: `event_ts + 1 second`.
- **Logic**: Simulates periodic inventory checks, with occasional low-stock scenarios at `STAND_002` to trigger alerts or restocking scenarios.

##### c. **Queue Events**
- **Fields**:
  - `event_id`: `EVT_QUEUE_{iteration}_<sequence>`.
  - `event_type`: `'queue'`.
  - `event_ts`: Random within 10 seconds.
  - `stand_id`: Randomly `STAND_001`, `STAND_002`, or `STAND_005` (`SEQ8() % 3`).
  - `sensor_id`: Maps to `stand_id` (e.g., `STAND_001` → `SENSOR_001`).
  - `payload`:
    - `queue_len`: Normally 5–15 people, but 20–30 for `STAND_001` every 5th iteration to simulate congestion.
  - `ingestion_time`: `event_ts + 1 second`.
- **Logic**: Simulates queue lengths at specific stands, with periodic spikes at `STAND_001` to mimic busy periods.

##### d. **Environmental Events**
- **Fields**:
  - `event_id`: `EVT_ENV_{iteration}_<sequence>`.
  - `event_type`: `'env'`.
  - `event_ts`: Random within 10 seconds.
  - `stand_id`: `NULL`.
  - `sensor_id`: Randomly `SENSOR_001` to `SENSOR_005`.
  - `payload`:
    - `temp_f`: Normally 68.0–77.0°F, but 80.6–89.6°F for `SENSOR_003` every 5th iteration to simulate high temperatures.
    - `co2_ppm`: Normally 600–1000 ppm, but 1000–1600 ppm for `SENSOR_002` every 5th iteration to simulate poor air quality.
    - `zone`: Maps to `sensor_id` (e.g., `SENSOR_001` → `West Concourse`).
  - `ingestion_time`: `event_ts + 1 second`.
- **Logic**: Simulates environmental sensor readings, with occasional spikes in temperature or CO2 to trigger maintenance alerts.

##### e. **Clean Request Events**
- **Fields**:
  - `event_id`: `EVT_CLEAN_{iteration}_<sequence>`.
  - `event_type`: `'clean_req'`.
  - `event_ts`: Random within 10 seconds.
  - `stand_id`, `sensor_id`: `NULL`.
  - `payload`:
    - `zone`: Randomly one of five zones (`West Concourse`, `East Concourse`, etc.).
    - `severity`: Randomly `warning` or `critical`.
    - `description`: Randomly `Spill reported near seating area` or `Restroom maintenance needed`.
  - `ingestion_time`: `event_ts + 1 second`.
- **Logic**: Simulates cleaning requests across the stadium, with varying urgency and location.

#### 4. **Execution Flow**
1. Retrieves the max `event_ts` and current `iteration`.
2. Calculates the `start_ts` for the current iteration.
3. Generates random row counts for each event type.
4. Constructs and executes the `INSERT` SQL statement.
5. Updates `ITERATION_TRACKER` with the new iteration and timestamp.
6. Returns a success message with the total rows inserted.

#### 5. **Key Features**
- **Temporal Progression**: Events are timestamped to simulate real-time activity, advancing by 10 seconds per iteration.
- **Randomization**: Varies row counts, timestamps, quantities, and metrics to mimic realistic fluctuations.
- **Conditional Logic**: Introduces spikes (e.g., low inventory, long queues, high temperature) periodically to simulate actionable scenarios.
- **Modularity**: Each event type is generated independently, allowing easy customization.
- **Scalability**: Uses Snowflake’s `GENERATOR` function for efficient row creation.

#### 6. **Use Case**
This procedure is designed for a demonstration simulating stadium operations, generating data for:
- Sales tracking (e.g., food and merchandise purchases).
- Inventory management (e.g., stock levels).
- Queue monitoring (e.g., wait times).
- Environmental conditions (e.g., temperature, air quality).
- Facility maintenance (e.g., cleaning requests).
The synthetic data supports testing downstream analytics, such as dynamic tables (`DT_CLEAN_EVENTS`, `DT_INVENTORY_STATUS`) for real-time insights.

#### 7. **Potential Enhancements**
- **Correlation**: Link sale and inventory events to reflect stock depletion (e.g., sales reduce `units_remaining`).
- **Event Dependencies**: Make queue lengths depend on sale volume or cleaning requests depend on environmental data.
- **Configurability**: Allow parameters for row counts, time windows, or spike frequencies.
- **Error Handling**: Add validation for `start_ts` or row counts to prevent overlaps or excessive data.

This procedure effectively generates diverse, realistic synthetic data for a stadium simulation, with built-in variability to demonstrate dynamic operational scenarios.

In [ ]:
-- Create a table to store the iteration count
CREATE TABLE IF NOT EXISTS ITERATION_TRACKER (
    iteration INT,
    last_updated TIMESTAMP_NTZ
);

-- Initialize the iteration count
INSERT INTO ITERATION_TRACKER (iteration, last_updated)
VALUES (0, CURRENT_TIMESTAMP());

-- Stored procedure for a single iteration
CREATE OR REPLACE PROCEDURE INSERT_ARENA_EVENTS_SINGLE()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python')
HANDLER = 'main'
AS
$$
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import StringType
from datetime import datetime, timedelta
import snowflake.snowpark as snowpark
import random

def main(session):
    try:
        max_ts_df = session.sql("SELECT MAX(event_ts) AS max_ts FROM RAW_EVENTS").collect()
        max_ts = max_ts_df[0]['MAX_TS'] if max_ts_df[0]['MAX_TS'] else datetime.strptime('2025-05-20 12:00:00', '%Y-%m-%d %H:%M:%S')
    except:
        max_ts = datetime.strptime('2025-05-20 12:00:00', '%Y-%m-%d %H:%M:%S')

    iteration_df = session.sql("SELECT iteration FROM ITERATION_TRACKER").collect()
    iteration = iteration_df[0]['ITERATION'] + 1

    start_ts = max_ts + timedelta(seconds=((iteration - 1) * 10))

    sale_rows = random.randint(8, 16)
    inv_rows = random.randint(6, 8)
    queue_rows = random.randint(5, 6)
    env_rows = random.randint(5, 6)
    clean_rows = random.randint(1, 2)
    total_rows = sale_rows + inv_rows + queue_rows + env_rows + clean_rows

    insert_sql = f"""
    INSERT INTO RAW_EVENTS
    SELECT
      event_id,
      event_type,
      event_ts,
      stand_id,
      sensor_id,
      payload,
      ingestion_time
    FROM (
      -- Sale events ({sale_rows} rows)
      SELECT
        'EVT_SALE_{iteration}_' || SEQ8() AS event_id,
        'sale' AS event_type,
        DATEADD(SECOND, UNIFORM(0, 9, RANDOM()), TO_TIMESTAMP_NTZ('{start_ts.strftime('%Y-%m-%d %H:%M:%S')}')) AS event_ts,
        CASE
          WHEN SEQ8() % 5 = 0 THEN 'STAND_001'
          WHEN SEQ8() % 5 = 1 THEN 'STAND_002'
          WHEN SEQ8() % 5 = 2 THEN 'STAND_003'
          WHEN SEQ8() % 5 = 3 THEN 'STAND_004'
          ELSE 'STAND_005'
        END AS stand_id,
        NULL AS sensor_id,
        OBJECT_CONSTRUCT(
          'sku_id', CASE
            WHEN stand_id IN ('STAND_001', 'STAND_004') THEN 'SKU_00' || (1 + (SEQ8() % 5))
            WHEN stand_id IN ('STAND_002', 'STAND_005') THEN 'SKU_00' || (6 + (SEQ8() % 5))
            ELSE 'SKU_0' || (11 + (SEQ8() % 5))
          END,
          'item_name', CASE
            WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE (1 + (SEQ8() % 5))
                WHEN 1 THEN 'Cheeseburger'
                WHEN 2 THEN 'Hot Dog'
                WHEN 3 THEN 'Nachos'
                WHEN 4 THEN 'Pizza Slice'
                ELSE 'Fries'
              END
            WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE (6 + (SEQ8() % 5))
                WHEN 6 THEN 'Draft Beer 16oz'
                WHEN 7 THEN 'Soda 20oz'
                WHEN 8 THEN 'Bottled Water'
                WHEN 9 THEN 'Iced Tea'
                ELSE 'Coffee'
              END
            ELSE CASE (11 + (SEQ8() % 5))
                WHEN 11 THEN 'Team Jersey'
                WHEN 12 THEN 'Cap'
                WHEN 13 THEN 'T-Shirt'
                WHEN 14 THEN 'Scarf'
                ELSE 'Keychain'
              END
          END,
          'vendor_name', CASE stand_id
            WHEN 'STAND_001' THEN 'Burger Bonanza'
            WHEN 'STAND_002' THEN 'Thirsty Fan Brews'
            WHEN 'STAND_003' THEN 'Team Spirit Merch'
            WHEN 'STAND_004' THEN 'Pizza Pavilion'
            ELSE 'Sip & Go'
          END,
          'units', CASE
            WHEN {iteration} % 10 = 0 THEN UNIFORM(5, 10, RANDOM())
            ELSE UNIFORM(1, 5, RANDOM())
          END,
          'price', CASE
            WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE (1 + (SEQ8() % 5))
                WHEN 1 THEN 10.0
                WHEN 2 THEN 8.0
                WHEN 3 THEN 7.0
                WHEN 4 THEN 6.0
                ELSE 5.0
              END
            WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE (6 + (SEQ8() % 5))
                WHEN 6 THEN 8.0
                WHEN 7 THEN 4.0
                WHEN 8 THEN 3.0
                WHEN 9 THEN 4.5
                ELSE 5.0
              END
            ELSE CASE (11 + (SEQ8() % 5))
                WHEN 11 THEN 50.0
                WHEN 12 THEN 25.0
                WHEN 13 THEN 20.0
                WHEN 14 THEN 15.0
                ELSE 5.0
              END
          END
        ) AS payload,
        DATEADD(SECOND, 1, event_ts) AS ingestion_time
      FROM TABLE(GENERATOR(ROWCOUNT => {sale_rows}))
      -- Inventory events ({inv_rows} rows)
      UNION ALL
      SELECT
        'EVT_INV_{iteration}_' || SEQ8() AS event_id,
        'inventory' AS event_type,
        DATEADD(SECOND, UNIFORM(0, 9, RANDOM()), TO_TIMESTAMP_NTZ('{start_ts.strftime('%Y-%m-%d %H:%M:%S')}')) AS event_ts,
        CASE
          WHEN SEQ8() % 5 = 0 THEN 'STAND_001'
          WHEN SEQ8() % 5 = 1 THEN 'STAND_002'
          WHEN SEQ8() % 5 = 2 THEN 'STAND_003'
          WHEN SEQ8() % 5 = 3 THEN 'STAND_004'
          ELSE 'STAND_005'
        END AS stand_id,
        NULL AS sensor_id,
        OBJECT_CONSTRUCT(
          'sku_id', CASE
            WHEN stand_id IN ('STAND_001', 'STAND_004') THEN 'SKU_00' || (1 + (SEQ8() % 5))
            WHEN stand_id IN ('STAND_002', 'STAND_005') THEN 'SKU_00' || (6 + (SEQ8() % 5))
            ELSE 'SKU_0' || (11 + (SEQ8() % 5))
          END,
          'item_name', CASE
            WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE (1 + (SEQ8() % 5))
                WHEN 1 THEN 'Cheeseburger'
                WHEN 2 THEN 'Hot Dog'
                WHEN 3 THEN 'Nachos'
                WHEN 4 THEN 'Pizza Slice'
                ELSE 'Fries'
              END
            WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE (6 + (SEQ8() % 5))
                WHEN 6 THEN 'Draft Beer 16oz'
                WHEN 7 THEN 'Soda 20oz'
                WHEN 8 THEN 'Bottled Water'
                WHEN 9 THEN 'Iced Tea'
                ELSE 'Coffee'
              END
            ELSE CASE (11 + (SEQ8() % 5))
                WHEN 11 THEN 'Team Jersey'
                WHEN 12 THEN 'Cap'
                WHEN 13 THEN 'T-Shirt'
                WHEN 14 THEN 'Scarf'
                ELSE 'Keychain'
              END
          END,
          'vendor_name', CASE stand_id
            WHEN 'STAND_001' THEN 'Burger Bonanza'
            WHEN 'STAND_002' THEN 'Thirsty Fan Brews'
            WHEN 'STAND_003' THEN 'Team Spirit Merch'
            WHEN 'STAND_004' THEN 'Pizza Pavilion'
            ELSE 'Sip & Go'
          END,
          'units_remaining', CASE
            WHEN stand_id = 'STAND_002' AND SEQ8() % 3 = 0 AND {iteration} % 5 = 0 THEN UNIFORM(50, 75, RANDOM()) * 0.01 * CASE (6 + (SEQ8() % 5))
                WHEN 6 THEN 450
                WHEN 7 THEN 375
                WHEN 8 THEN 600
                WHEN 9 THEN 300
                ELSE 225
              END
            ELSE UNIFORM(75, 100, RANDOM()) * 0.01 * CASE
                WHEN stand_id IN ('STAND_001', 'STAND_004') THEN CASE (1 + (SEQ8() % 5))
                    WHEN 1 THEN 150
                    WHEN 2 THEN 200
                    WHEN 3 THEN 120
                    WHEN 4 THEN 180
                    ELSE 300
                  END
                WHEN stand_id IN ('STAND_002', 'STAND_005') THEN CASE (6 + (SEQ8() % 5))
                    WHEN 6 THEN 450
                    WHEN 7 THEN 375
                    WHEN 8 THEN 600
                    WHEN 9 THEN 300
                    ELSE 225
                  END
                ELSE CASE (11 + (SEQ8() % 5))
                    WHEN 11 THEN 75
                    WHEN 12 THEN 110
                    WHEN 13 THEN 150
                    WHEN 14 THEN 90
                    ELSE 300
                  END
              END
          END
        ) AS payload,
        DATEADD(SECOND, 1, event_ts) AS ingestion_time
      FROM TABLE(GENERATOR(ROWCOUNT => {inv_rows}))
      -- Queue events ({queue_rows} rows)
      UNION ALL
      SELECT
        'EVT_QUEUE_{iteration}_' || SEQ8() AS event_id,
        'queue' AS event_type,
        DATEADD(SECOND, UNIFORM(0, 9, RANDOM()), TO_TIMESTAMP_NTZ('{start_ts.strftime('%Y-%m-%d %H:%M:%S')}')) AS event_ts,
        CASE
          WHEN SEQ8() % 3 = 0 THEN 'STAND_001'
          WHEN SEQ8() % 3 = 1 THEN 'STAND_002'
          ELSE 'STAND_005'
        END AS stand_id,
        CASE
          WHEN stand_id = 'STAND_001' THEN 'SENSOR_001'
          WHEN stand_id = 'STAND_002' THEN 'SENSOR_002'
          ELSE 'SENSOR_005'
        END AS sensor_id,
        OBJECT_CONSTRUCT(
          'queue_len', CASE
            WHEN stand_id = 'STAND_001' AND {iteration} % 5 = 0 THEN UNIFORM(20, 30, RANDOM())
            ELSE UNIFORM(5, 15, RANDOM())
          END
        ) AS payload,
        DATEADD(SECOND, 1, event_ts) AS ingestion_time
      FROM TABLE(GENERATOR(ROWCOUNT => {queue_rows}))
      -- Environmental events ({env_rows} rows)
      UNION ALL
      SELECT
        'EVT_ENV_{iteration}_' || SEQ8() AS event_id,
        'env' AS event_type,
        DATEADD(SECOND, UNIFORM(0, 9, RANDOM()), TO_TIMESTAMP_NTZ('{start_ts.strftime('%Y-%m-%d %H:%M:%S')}')) AS event_ts,
        NULL AS stand_id,
        CASE
          WHEN SEQ8() % 5 = 0 THEN 'SENSOR_001'
          WHEN SEQ8() % 5 = 1 THEN 'SENSOR_002'
          WHEN SEQ8() % 5 = 2 THEN 'SENSOR_003'
          WHEN SEQ8() % 5 = 3 THEN 'SENSOR_004'
          ELSE 'SENSOR_005'
        END AS sensor_id,
        OBJECT_CONSTRUCT(
          'temp_f', CASE
            WHEN sensor_id = 'SENSOR_003' AND {iteration} % 5 = 0 THEN ROUND(UNIFORM(80.6, 89.6, RANDOM()), 1)
            ELSE ROUND(UNIFORM(68.0, 77.0, RANDOM()), 1)
          END,
          'co2_ppm', CASE
            WHEN sensor_id = 'SENSOR_002' AND {iteration} % 5 = 0 THEN UNIFORM(1000, 1600, RANDOM())
            ELSE UNIFORM(600, 1000, RANDOM())
          END,
          'zone', CASE
            WHEN sensor_id = 'SENSOR_001' THEN 'West Concourse'
            WHEN sensor_id = 'SENSOR_002' THEN 'East Concourse'
            WHEN sensor_id = 'SENSOR_003' THEN 'Lower Bowl A'
            WHEN sensor_id = 'SENSOR_004' THEN 'Club Seats North'
            ELSE 'Upper Deck South'
          END
        ) AS payload,
        DATEADD(SECOND, 1, event_ts) AS ingestion_time
      FROM TABLE(GENERATOR(ROWCOUNT => {env_rows}))
      -- Clean request events ({clean_rows} rows)
      UNION ALL
      SELECT
        'EVT_CLEAN_{iteration}_' || SEQ8() AS event_id,
        'clean_req' AS event_type,
        DATEADD(SECOND, UNIFORM(0, 9, RANDOM()), TO_TIMESTAMP_NTZ('{start_ts.strftime('%Y-%m-%d %H:%M:%S')}')) AS event_ts,
        NULL AS stand_id,
        NULL AS sensor_id,
        OBJECT_CONSTRUCT(
          'zone', CASE
            WHEN SEQ8() % 5 = 0 THEN 'West Concourse'
            WHEN SEQ8() % 5 = 1 THEN 'East Concourse'
            WHEN SEQ8() % 5 = 2 THEN 'Lower Bowl A'
            WHEN SEQ8() % 5 = 3 THEN 'Club Seats North'
            ELSE 'Upper Deck South'
          END,
          'severity', CASE
            WHEN SEQ8() % 2 = 0 THEN 'warning'
            ELSE 'critical'
          END,
          'description', CASE
            WHEN SEQ8() % 2 = 0 THEN 'Spill reported near seating area'
            ELSE 'Restroom maintenance needed'
          END
        ) AS payload,
        DATEADD(SECOND, 1, event_ts) AS ingestion_time
      FROM TABLE(GENERATOR(ROWCOUNT => {clean_rows}))
    )
    """

    session.sql(insert_sql).collect()
    session.sql(f"UPDATE iteration_tracker SET iteration = {iteration}, last_updated = CURRENT_TIMESTAMP()").collect()

    return f"Inserted {total_rows} rows for iteration {iteration}"
$$;

### 8. Introducing Dynamic Tables

**Objective**: Understand Dynamic Tables and their power in real-time data pipelines.

**What Are They?**: Snowflake’s Dynamic Tables use SQL to:
- Auto-refresh on upstream changes.
- Process with low latency (e.g., 1-min lag).
- Manage dependencies effortlessly.
- Scale with Snowflake’s cloud.

In ArenaFlow, they turn raw events into clean, aggregated KPIs for a live dashboard, handling:
- Cleaning raw data.
- Minute-by-minute sales, inventory, and queue metrics.
- Actionable alerts.

**Parameters**:
- **TARGET_LAG**: Max delay (e.g., seconds, ‘DOWNSTREAM’ for sync).
- **WAREHOUSE**: Compute resource for refreshes.
- **REFRESH_MODE**: AUTO, FULL, or INCREMENTAL.
- **INITIALIZE**: ON_CREATE or ON_SCHEDULE.
- **COMMENT**: Add notes for clarity.
- **CLUSTER BY**: Boosts query speed.
- **DATA_RETENTION**: Time travel duration.
- **ROW ACCESS POLICY**: Secures data access.

**Value**: Simplifies streaming, delivering fast insights. Spot a 20-person queue in <1 min, dispatch staff, and boost sales—1% better queues can add thousands per game.

**Next**: Build the first Dynamic Table for cleaning.

### 9. Dynamic Table: DT_CLEAN_EVENTS

**Objective**: Normalize raw JSON into structured, query-ready rows.

**Purpose**: This staging table parses `RAW_EVENTS` payloads, buckets by `minute_ts`, and validates fields for consistency.

**Depends On**: `RAW_EVENTS`

**Key Outputs**:
- `event_id`, `event_type`, `minute_ts`, `event_ts`, `stand_id`, `sensor_id`
- `is_valid`: Checks required fields per event type.
- Parsed: `sku_id`, `units`, `price`, `queue_len`, `temp_f`, `co2_ppm`, `units_remaining`, `zone`, `severity`, `description`

**Value**: Ensures reliable, structured data for all downstream metrics—sales, inventory, queues, and alerts—enabling accurate, fast analysis.

**Role**: Foundation for all operational tables in the pipeline.

**Next**: Verify and explore sales metrics.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_CLEAN_EVENTS
LAG = '1 minute'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
AS
SELECT
    event_id,
    event_type,
    DATE_TRUNC('MINUTE', event_ts) AS minute_ts,
    event_ts,
    stand_id,
    sensor_id,
    CASE
        WHEN event_type = 'sale' AND payload:sku_id IS NOT NULL AND payload:units IS NOT NULL AND payload:price IS NOT NULL THEN TRUE
        WHEN event_type = 'inventory' AND payload:sku_id IS NOT NULL AND payload:units_remaining IS NOT NULL THEN TRUE
        WHEN event_type = 'queue' AND payload:queue_len IS NOT NULL THEN TRUE
        WHEN event_type = 'env' AND payload:zone IS NOT NULL AND (payload:temp_f IS NOT NULL OR payload:co2_ppm IS NOT NULL) THEN TRUE
        WHEN event_type = 'clean_req' AND payload:zone IS NOT NULL AND payload:severity IS NOT NULL THEN TRUE
        ELSE FALSE
    END AS is_valid,
    payload:sku_id::STRING AS sku_id,
    payload:item_name::STRING AS item_name,
    payload:vendor_name::STRING AS vendor_name,
    payload:units::INTEGER AS units,
    payload:price::FLOAT AS price,
    payload:queue_len::INTEGER AS queue_len,
    payload:temp_f::FLOAT AS temp_f,
    payload:co2_ppm::FLOAT AS co2_ppm,
    payload:units_remaining::INTEGER AS units_remaining,
    payload:zone::STRING AS zone,
    payload:severity::STRING AS severity,
    payload:description::STRING AS description,
    ingestion_time
FROM RAW_EVENTS
WHERE payload IS NOT NULL;

In [ ]:
SHOW DYNAMIC TABLES IN SCHEMA DYNAMIC_TABLES;

In [ ]:
-- QA
SELECT *
FROM DT_CLEAN_EVENTS;

### 10. Dynamic Table: DT_SALES_MINUTE & DT_SALES_AGG
**Place above cells: "DT_SALES_MINUTE_1" (id: d526db6c-1b17-4067-ac96-bd74a0996a79) through "QA_DT_SALES_MINUTE_2" (id: 95d605bc-5527-4c2f-af70-cd0b922dc341)**

**Objective**: Track real-time sales activity per stand.

**Purpose**:
- **DT_SALES_AGG**: Aggregates valid sales from `DT_CLEAN_EVENTS` by minute and stand, calculating revenue and units sold.
- **DT_SALES_MINUTE**: Enriches aggregates with stand details, identifying top-selling SKUs, items, and vendors.

**Depends On**:
- `DT_CLEAN_EVENTS`
- `Stand Directory`

**Key Outputs**:
- `minute_ts`, `stand_id`, `category`, `vendor_name`
- `revenue` (FLOAT), `units_sold` (INTEGER)
- `top_sku`, `top_item_name`, `top_vendor_name`

**Value**: Reveals sales trends and hot products minute-by-minute, guiding staff to high-demand areas and boosting revenue.

**Role**: Feeds the Arena Dashboard and tracks inventory depletion.

**Next**: Monitor inventory levels.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_SALES_MINUTE
LAG = '70 Seconds'
REFRESH_MODE = 'INCREMENTAL'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
AS
WITH sales_agg AS (
    SELECT
        minute_ts,
        stand_id,
        SUM(units * price) AS revenue,
        SUM(units) AS units_sold,
        MAX_BY(sku_id, units) AS top_sku,
        MAX_BY(item_name, units) AS top_item_name,
        MAX_BY(vendor_name, units) AS top_vendor_name
    FROM dt_clean_events
    WHERE event_type = 'sale' AND is_valid = TRUE
    GROUP BY minute_ts, stand_id
)
SELECT
    s.minute_ts,
    s.stand_id,
    d.category,
    d.location AS vendor_name,
    s.revenue,
    s.units_sold,
    s.top_sku,
    s.top_item_name,
    s.top_vendor_name
FROM sales_agg s
JOIN stand_directory d ON s.stand_id = d.stand_id;

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_SALES_AGG
LAG = 'DOWNSTREAM'
REFRESH_MODE = 'INCREMENTAL'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
AS
SELECT
    minute_ts,
    stand_id,
    SUM(units * price) AS revenue,
    SUM(units) AS units_sold
FROM dt_clean_events
WHERE event_type = 'sale' AND is_valid = TRUE
GROUP BY minute_ts, stand_id;

In [ ]:
SHOW DYNAMIC TABLES IN SCHEMA DYNAMIC_TABLES;

In [ ]:
SELECT *
FROM DT_SALES_AGG
ORDER BY 1;

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_SALES_MINUTE
LAG = '70 Seconds'
REFRESH_MODE = 'INCREMENTAL'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
AS
SELECT
    agg.minute_ts,
    agg.stand_id,
    d.category,
    d.location AS vendor_name,
    agg.revenue,
    agg.units_sold,
    r.top_sku,
    r.top_item_name,
    r.top_vendor_name
FROM DT_SALES_AGG AS agg
JOIN stand_directory d ON agg.stand_id = d.stand_id
JOIN (
    SELECT 
        minute_ts,
        stand_id,
        sku_id AS top_sku,
        item_name AS top_item_name,
        vendor_name AS top_vendor_name,
        ROW_NUMBER() OVER (PARTITION BY minute_ts, stand_id ORDER BY units DESC) AS rn
    FROM dt_clean_events
    WHERE event_type = 'sale' AND is_valid = TRUE
) AS r ON agg.minute_ts = r.minute_ts AND agg.stand_id = r.stand_id
WHERE r.rn = 1;

In [ ]:
SELECT *
FROM DT_SALES_MINUTE
ORDER BY 1;

### 11. Dynamic Table: DT_INVENTORY_STATUS

**Objective**: Monitor stock levels and flag low inventory.

**Purpose**: Processes sensor data to track remaining units per SKU and stand, calculating stock percentage against par levels.

**Depends On**:
- `DT_CLEAN_EVENTS`
- `Product Catalog`

**Key Outputs**:
- `minute_ts`, `stand_id`, `sku_id`, `item_name`, `vendor_name`
- `units_remaining` (INTEGER)
- `stock_pct` (FLOAT)

**Value**: Spots low stock early, enabling restocking before fans miss out, protecting sales and satisfaction.

**Role**: Drives the Ops Alerts system for stock-out warnings.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_INVENTORY_STATUS
LAG = '70 Seconds'
REFRESH_MODE = 'INCREMENTAL'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
CLUSTER BY (vendor_name)
AS
WITH RankedInventory AS (
    SELECT
        e.minute_ts,
        e.stand_id,
        e.sku_id,
        e.item_name,
        e.vendor_name,
        e.units_remaining,
        ROW_NUMBER() OVER (
            PARTITION BY e.minute_ts, e.stand_id, e.sku_id, e.item_name, e.vendor_name
            ORDER BY e.event_ts DESC, e.ingestion_time DESC
        ) AS rn
    FROM DT_CLEAN_EVENTS e
    JOIN PRODUCT_CATALOG p ON e.sku_id = p.sku_id
    WHERE e.event_type = 'inventory' 
      AND e.is_valid = TRUE
)
SELECT
    minute_ts,
    stand_id,
    sku_id,
    item_name,
    vendor_name,
    units_remaining,
    (units_remaining / CASE sku_id
        WHEN 'SKU_001' THEN 150
        WHEN 'SKU_002' THEN 200
        WHEN 'SKU_003' THEN 120
        WHEN 'SKU_004' THEN 180
        WHEN 'SKU_005' THEN 300
        WHEN 'SKU_006' THEN 450
        WHEN 'SKU_007' THEN 375
        WHEN 'SKU_008' THEN 600
        WHEN 'SKU_009' THEN 300
        WHEN 'SKU_010' THEN 225
        WHEN 'SKU_011' THEN 75
        WHEN 'SKU_012' THEN 110
        WHEN 'SKU_013' THEN 150
        WHEN 'SKU_014' THEN 90
        WHEN 'SKU_015' THEN 300
    END) AS stock_pct
FROM RankedInventory
WHERE rn = 1
  AND units_remaining IS NOT NULL;

In [ ]:
SHOW DYNAMIC TABLES IN SCHEMA DYNAMIC_TABLES;

In [ ]:
-- QA
SELECT *
FROM DT_INVENTORY_STATUS
ORDER BY 1;

### 12. Dynamic Table: DT_QUEUE_METRICS

**Objective**: Measure queue lengths and trends at stands.

**Purpose**: Uses camera and sensor data to average queue lengths per minute and track 5-minute trends for early congestion detection.

**Depends On**:
- `DT_CLEAN_EVENTS`
- `Stand Directory`

**Key Outputs**:
- `minute_ts`, `stand_id`, `category`, `vendor_name`
- `queue_len` (INTEGER)
- `trend_5min` (FLOAT)

**Value**: Identifies long lines fast, allowing staff redeployment to cut wait times and lift concession throughput.

**Role**: Fuels the alerting system for line warnings.

**Next**: Track environmental comfort.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_QUEUE_METRICS
LAG = '70 Seconds'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
REFRESH_MODE = 'INCREMENTAL'
AS
WITH queue_data AS (
    SELECT
        minute_ts,
        stand_id,
        AVG(queue_len) AS queue_len
    FROM dt_clean_events
    WHERE event_type = 'queue' AND is_valid = TRUE
    GROUP BY minute_ts, stand_id
),
trend_calc AS (
    SELECT
        q.minute_ts,
        q.stand_id,
        q.queue_len,
        (q.queue_len - LAG(q.queue_len, 5) OVER (PARTITION BY q.stand_id ORDER BY q.minute_ts)) / 5.0 AS trend_5min
    FROM queue_data q
)
SELECT
    t.minute_ts,
    t.stand_id,
    d.category,
    d.location AS vendor_name,
    t.queue_len,
    t.trend_5min
FROM trend_calc t
JOIN stand_directory d ON t.stand_id = d.stand_id;

In [ ]:
-- QA
SELECT *
FROM DT_QUEUE_METRICS
ORDER BY 1;

### 13. Dynamic Table: DT_ENV_COMFORT

**Objective**: Monitor temperature and air quality for fan comfort.

**Purpose**: Aggregates sensor data to summarize average temperature and max CO2 levels per zone, minute-by-minute.

**Depends On**:
- `DT_CLEAN_EVENTS`
- `Sensor Map`

**Key Outputs**:
- `minute_ts`, `zone`
- `avg_temp_f` (FLOAT)
- `max_co2_ppm` (FLOAT)

**Value**: Ensures a comfortable fan experience by flagging hot or stuffy zones for HVAC adjustments.

**Role**: Feeds Ops Alerts for environmental fixes.

**Next**: Consolidate alerts for action.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_ENV_COMFORT
LAG = '70 Seconds'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
REFRESH_MODE = 'INCREMENTAL'
AS
SELECT
    e.minute_ts,
    e.zone,
    AVG(e.temp_f) AS avg_temp_f,
    MAX(e.co2_ppm) AS max_co2_ppm
FROM dt_clean_events e
JOIN sensor_map s ON e.sensor_id = s.sensor_id
WHERE e.event_type = 'env' AND e.is_valid = TRUE
GROUP BY e.minute_ts, e.zone
HAVING avg_temp_f IS NOT NULL OR max_co2_ppm IS NOT NULL;

In [ ]:
-- QA
SELECT *
FROM DT_ENV_COMFORT
ORDER BY 1

### 14. Dynamic Table: DT_OPS_ALERTS

**Objective**: Aggregate actionable alerts for staff.

**Purpose**: Combines thresholds from `Maintenance Thresholds` to flag:
- **Low Stock**: `stock_pct` below limits.
- **Long Lines**: `queue_len` or trend exceeds bounds.
- **Environmental Issues**: High temp or CO2 levels.

**Depends On**:
- `DT_INVENTORY_STATUS`
- `DT_QUEUE_METRICS`
- `DT_ENV_COMFORT`
- `Maintenance Thresholds`

**Key Outputs**:
- `alert_id`, `alert_ts`, `stand_or_zone`
- `metric_type`, `current_value`, `severity`
- `status` (open/closed), `action_required`

**Value**: Prioritizes urgent issues—restocking, staff adds, HVAC tweaks—for fast resolution, keeping operations smooth.

**Role**: Delivers a clear action list for staff.

**Next**: Build the dashboard view.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_OPS_ALERTS
LAG = '70 Seconds'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
REFRESH_MODE = 'INCREMENTAL'
AS
WITH stock_alerts AS (
    SELECT
        minute_ts AS alert_ts,
        CONCAT('ALERT_STOCK_', stand_id, '_', sku_id, '_', minute_ts) AS alert_id,
        stand_id AS stand_or_zone,
        'stock_pct' AS metric_type,
        stock_pct AS current_value,
        m.severity,
        CASE
            WHEN ABS(MOD(HASH(alert_id), 3)) = 0 THEN 'closed'
            ELSE 'open'
        END AS status,
        'Restock ' || item_name || ' at ' || vendor_name AS action_required
    FROM dt_inventory_status i
    JOIN maintenance_thresholds m ON m.metric_type = 'stock_pct' AND i.stock_pct <= m.threshold_value
),
queue_alerts AS (
    SELECT
        minute_ts AS alert_ts,
        CONCAT('ALERT_QUEUE_', stand_id, '_', minute_ts) AS alert_id,
        stand_id AS stand_or_zone,
        'queue_len' AS metric_type,
        queue_len AS current_value,
        m.severity,
        CASE
            WHEN ABS(MOD(HASH(alert_id), 3)) = 0 THEN 'closed'
            ELSE 'open'
        END AS status,
        'Add staff to ' || vendor_name || ' to reduce queue' AS action_required
    FROM dt_queue_metrics q
    JOIN maintenance_thresholds m ON m.metric_type = 'queue_len' AND q.queue_len >= m.threshold_value
),
env_alerts AS (
    SELECT
        minute_ts AS alert_ts,
        CONCAT('ALERT_ENV_', zone, '_', e.metric_type, '_', minute_ts) AS alert_id,
        zone AS stand_or_zone,
        e.metric_type,
        current_value,
        m.severity,
        CASE
            WHEN ABS(MOD(HASH(alert_id), 3)) = 0 THEN 'closed'
            ELSE 'open'
        END AS status,
        CASE e.metric_type
            WHEN 'temp_f' THEN 'Adjust HVAC in ' || zone
            WHEN 'co2_ppm' THEN 'Increase ventilation in ' || zone
        END AS action_required
    FROM (
        SELECT
            minute_ts,
            zone,
            'temp_f' AS metric_type,
            avg_temp_f AS current_value
        FROM dt_env_comfort
        UNION ALL
        SELECT
            minute_ts,
            zone,
            'co2_ppm' AS metric_type,
            max_co2_ppm AS current_value
        FROM dt_env_comfort
    ) e
    JOIN maintenance_thresholds m ON e.metric_type = m.metric_type AND e.current_value >= m.threshold_value
)
SELECT * FROM stock_alerts
UNION ALL
SELECT * FROM queue_alerts
UNION ALL
SELECT * FROM env_alerts;

In [ ]:
-- QA
SELECT *
FROM DT_OPS_ALERTS
ORDER BY 1;

### 15. Dynamic Table: DT_ARENA_DASHBOARD

**Objective**: Unify real-time KPIs for arena oversight.

**Purpose**: Combines sales, queues, alerts, and comfort metrics into a single view for managers, highlighting key trends and issues.

**Depends On**:
- `DT_SALES_MINUTE`
- `DT_OPS_ALERTS`
- `DT_QUEUE_METRICS`
- `DT_ENV_COMFORT`
- `DT_INVENTORY_STATUS`

**Key Outputs**:
- `minute_ts`
- `total_units_sold`, `total_revenue`
- `avg_queue_len`, `open_alerts`
- `hottest_zone`, `worst_stockout`

**Value**: Powers a command center view, driving fast decisions to optimize sales, reduce waits, and enhance comfort—potentially adding thousands in revenue.

**Role**: Final hub for BI tools, signage, and tablet dashboards.

**Next**: Simulate ongoing data and test.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE DT_ARENA_DASHBOARD
LAG = '90 Seconds'
WAREHOUSE = 'DEV_WH_ARENAFLOW_HOLS'
AS
WITH sales_summary AS (
    SELECT
        minute_ts,
        SUM(units_sold) AS total_units_sold,
        SUM(revenue) AS total_revenue
    FROM dt_sales_minute
    GROUP BY minute_ts
),
queue_summary AS (
    SELECT
        minute_ts,
        AVG(queue_len) AS avg_queue_len
    FROM dt_queue_metrics
    GROUP BY minute_ts
),
alert_summary AS (
    SELECT
        alert_ts AS minute_ts,
        COUNT(*) AS open_alerts
    FROM dt_ops_alerts
    WHERE status = 'open'
    GROUP BY alert_ts
),
hottest_zone AS (
    SELECT
        minute_ts,
        MAX_BY(zone, avg_temp_f) AS hottest_zone
    FROM dt_env_comfort
    GROUP BY minute_ts
),
worst_stockout AS (
    SELECT
        minute_ts,
        MIN_BY(CONCAT(vendor_name, ' (', item_name, ') | ', ROUND(stock_pct * 100, 1), '%'), stock_pct) AS worst_stockout
    FROM dt_inventory_status
    GROUP BY minute_ts
)
SELECT
    COALESCE(s.minute_ts, q.minute_ts, a.minute_ts, h.minute_ts, w.minute_ts) AS minute_ts,
    s.total_units_sold,
    s.total_revenue,
    q.avg_queue_len,
    a.open_alerts,
    h.hottest_zone,
    w.worst_stockout
FROM sales_summary s
FULL OUTER JOIN queue_summary q ON s.minute_ts = q.minute_ts
FULL OUTER JOIN alert_summary a ON s.minute_ts = a.minute_ts
FULL OUTER JOIN hottest_zone h ON s.minute_ts = h.minute_ts
FULL OUTER JOIN worst_stockout w ON s.minute_ts = w.minute_ts;

In [ ]:
-- QA
SELECT *
FROM DT_ARENA_DASHBOARD
ORDER BY 1;

### 16. Continuous Data Simulation

**Objective**: Run ongoing synthetic data generation.

**Context**: This Python script calls `INSERT_ARENA_EVENTS_SINGLE` in a loop, adding new events every 10 seconds to mimic live stadium data.

**Value**: Tests the pipeline’s real-time response, ensuring Dynamic Tables refresh and deliver insights as events flow, prepping for live games.

**Note**: Stop manually or set `max_iterations` for control.

**Next**: Suspend tables when done.

In [ ]:
from snowflake.snowpark.context import get_active_session
import time

session = get_active_session()

def run_continuous_insert(session, max_iterations=None):
    iteration_count = 0
    while max_iterations is None or iteration_count < max_iterations:
        # Call the stored procedure
        result = session.call("insert_arena_events_single")
        print(result)
        
        # Wait for 10 seconds
        time.sleep(10)
        iteration_count += 1

        
run_continuous_insert(session, max_iterations=None)

### 17. Suspend Dynamic Tables

**Objective**: Pause Dynamic Tables to save resources.

**Context**: This SQL suspends all Dynamic Tables—`DT_CLEAN_EVENTS`, `DT_SALES_MINUTE`, `DT_INVENTORY_STATUS`, `DT_QUEUE_METRICS`, `DT_ENV_COMFORT`, `DT_OPS_ALERTS`, `DT_ARENA_DASHBOARD`—halting refreshes.

**Value**: Conserves compute costs when testing or simulation is complete, maintaining efficiency.

**Note**: Resume with `ALTER DYNAMIC TABLE <name> RESUME` when ready.

In [ ]:
ALTER DYNAMIC TABLE DT_ENV_COMFORT SUSPEND;
ALTER DYNAMIC TABLE DT_CLEAN_EVENTS SUSPEND;
ALTER DYNAMIC TABLE DT_ARENA_DASHBOARD SUSPEND;
ALTER DYNAMIC TABLE DT_INVENTORY_STATUS SUSPEND;
ALTER DYNAMIC TABLE DT_OPS_ALERTS SUSPEND;
ALTER DYNAMIC TABLE DT_QUEUE_METRICS SUSPEND;
ALTER DYNAMIC TABLE DT_SALES_MINUTE SUSPEND;